In [6]:
from google.colab import drive
drive.mount('/content/drive')

!pip install "datasets>=2.5.2" "numpy>=1.24.2" "pandas>=1.4.4" "scikit-learn>=1.1.1" "torch>=1.10.2" "tqdm>=4.64.1" "transformers>=4.22.1"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/Tabula-main-claude

/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude


In [8]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula import Tabula
import pandas as pd

In [9]:
from sklearn.model_selection import train_test_split


# df = pd.read_csv("Real_Datasets/abalone.csv")

# sex_mapping = {'M': 1, 'F': 0, 'I': 2}

# df.iloc[:, 0] = df.iloc[:, 0].map(sex_mapping)

# output_path = 'Real_Datasets/abalone_number.csv'
# df.to_csv(output_path, index=False)

# train_data, test_data = train_test_split(df, test_size=0.22, random_state=42)

# train_data.to_csv("Real_Datasets/abalone-train.csv", index=False)
# test_data.to_csv("Real_Datasets/abalone-test.csv", index=False)

train_data = pd.read_csv("Real_Datasets/abalone-train.csv")

In [10]:
train_data

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,1,0.465,0.375,0.110,0.5000,0.2100,0.1130,0.1505,8
1,1,0.660,0.545,0.185,1.3200,0.5305,0.2635,0.4550,16
2,0,0.480,0.375,0.105,0.5250,0.2185,0.1195,0.1550,12
3,2,0.620,0.470,0.140,0.8565,0.3595,0.1600,0.2950,9
4,1,0.545,0.440,0.140,0.8395,0.3560,0.1905,0.2385,11
...,...,...,...,...,...,...,...,...,...
3253,0,0.490,0.400,0.115,0.5690,0.2560,0.1325,0.1450,9
3254,0,0.670,0.550,0.190,1.3905,0.5425,0.3035,0.4000,12
3255,1,0.510,0.395,0.125,0.5805,0.2440,0.1335,0.1880,11
3256,1,0.575,0.465,0.120,1.0535,0.5160,0.2185,0.2350,9


In [11]:
categorical_columns = ["sex", "rings"]
import torch

for sigma_value in [0.3, 0.35]:  
    model = Tabula(
        llm='distilgpt2',
        experiment_dir="adult_training",
        batch_size=64,
        epochs=100,
        categorical_columns=categorical_columns,
        use_dp=True,
        clip_coeff=1,
        sigma=sigma_value,
        micro_batch_size=16,
        learning_rate=5e-4
    )


    model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)

    model.fit(train_data)

    torch.save(model.model.state_dict(), "insurance_training/model_400epoch-lr-1e-4.pt")

    synthetic_data = model.sample(n_samples=3258)
    sigma_str = f"{sigma_value:.2f}".replace('.', '')
    synthetic_data.to_csv(f"abalone_syn_data/abalone-sigma-{sigma_str}.csv", index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

<ipython-input-11-b996f1549178>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)
/content

Step,Training Loss
500,3.289200
1000,1.704200
1500,1.593600
2000,1.546800
2500,1.515000
3000,1.492300
3500,1.478100
4000,1.463000
4500,1.451500
5000,1.443800


100%|██████████| 3258/3258 [13:08<00:00,  4.13it/s]
<ipython-input-11-b996f1549178>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load(

Step,Training Loss
500,3.501900
1000,1.779400
1500,1.643400
2000,1.589100
2500,1.552400
3000,1.525400
3500,1.507700
4000,1.490300
4500,1.476700
5000,1.468100


3260it [14:48,  3.67it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')


In [ ]:
categorical_columns = ["sex", "rings"]
import torch

for sigma_value in [0.2, 0.25, 0.3]:  
    model = Tabula(
        llm='distilgpt2',
        experiment_dir="adult_training",
        batch_size=64,
        epochs=100,
        categorical_columns=categorical_columns,
        use_dp=True,
        clip_coeff=1,
        sigma=sigma_value,
        micro_batch_size=16,
        learning_rate=5e-4
    )


    model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)

    model.fit(train_data)

    torch.save(model.model.state_dict(), "insurance_training/model_400epoch-lr-1e-4.pt")

    synthetic_data = model.sample(n_samples=3258)
    sigma_str = f"{sigma_value:.2f}".replace('.', '')
    synthetic_data.to_csv(f"abalone_syn_data/abalone-sigma-{sigma_str}.csv", index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Step,Training Loss
500,2.784700
1000,1.563000
1500,1.486600
2000,1.458200
2500,1.439400
3000,1.423900
3500,1.413400
4000,1.401900
4500,1.392400
5000,1.386400


100%|██████████| 3258/3258 [11:02<00:00,  4.92it/s]
<ipython-input-8-b4aaa3c73178>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("

Step,Training Loss
500,3.031100
1000,1.632500
1500,1.540000
2000,1.500500
2500,1.476700
3000,1.458600
3500,1.445900
4000,1.432700
4500,1.421900
5000,1.415100


3260it [12:28,  4.36it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
<ipython-input-8-b4aaa3c73178>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the fu

Step,Training Loss
500,3.300500
1000,1.705400
1500,1.593800
2000,1.546100
2500,1.514400
3000,1.492200
3500,1.477000
4000,1.461900
4500,1.450300


In [7]:
categorical_columns = ["sex", "rings"]
import torch

for sigma_value in [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2]: 
    model = Tabula(
        llm='distilgpt2',
        experiment_dir="adult_training",
        batch_size=64,
        epochs=50,
        categorical_columns=categorical_columns,
        use_dp=True,
        clip_coeff=1,
        sigma=sigma_value,
        micro_batch_size=16,
        learning_rate=5e-4
    )


    model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)

    model.fit(train_data)

    torch.save(model.model.state_dict(), "insurance_training/model_400epoch-lr-1e-4.pt")

    synthetic_data = model.sample(n_samples=3258)
    sigma_str = f"{sigma_value:.2f}".replace('.', '')
    synthetic_data.to_csv(f"abalone_syn_data/abalone-sigma-{sigma_str}.csv", index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-7-3c674b3637a4>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the 

Step,Training Loss
500,1.448800
1000,1.074300
1500,1.026300
2000,1.004000
2500,0.991500


  0%|          | 0/3258 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
3302it [00:16, 204.44it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
<ipython-input-7-3c674b3637a4>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malic

Step,Training Loss
500,1.560200
1000,1.178700
1500,1.114300
2000,1.083400
2500,1.068200


3307it [00:21, 155.68it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
<ipython-input-7-3c674b3637a4>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the f

...

Step,Training Loss
500,2.211600
1000,1.408100
1500,1.363600
2000,1.341300
2500,1.326900


100%|██████████| 3258/3258 [14:21<00:00,  3.78it/s]
<ipython-input-7-3c674b3637a4>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("

Step,Training Loss
500,2.531300
1000,1.490300
1500,1.422400
2000,1.394100
2500,1.378200


  2%|▏         | 66/3258 [00:31<25:44,  2.07it/s]


KeyboardInterrupt: 